In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sim.dy.model_intv import ModelIntv
import sim.dy.keys as I
import numpy.random as rd

In [2]:
n_collect = 5

In [3]:
def get_stats(ms, idx, **kwargs):
    ms_stats = dict()
    ms_stats.update(kwargs)
    ms_stats['Pop0'] = ms.Pop.iloc[0]
    
    for key in idx[1:]:
        try:
            ms_stats[key] = (ms[key] * ms.Pop).sum() * 0.5
        except KeyError:
            ms_stats[key] = 0
    return ms_stats

In [6]:
def collect_fux(pars_post, n_collect=20, coverage=0.03, duration=2, tp="VulACF"):
    idx = ['Pop', 'IncR', 'MorR', 
           'ACF_Vul_Footfall', 'ACF_Vul_Screened', 'ACF_Vul_Confirmed', 'ACF_Vul_Yield', 'ACF_Vul_TP',
           'ACF_Vul_DS_Fl', 'ACF_Vul_DR_Fl', 'ACF_Vul_DR_Sl', 
           'ACF_Vulfu_Footfall', 'ACF_Vulfu_Screened', 'ACF_Vulfu_Confirmed', 'ACF_Vulfu_Yield', 'ACF_Vulfu_TP',
           'ACF_Vulfu_DS_Fl', 'ACF_Vulfu_DR_Fl', 'ACF_Vulfu_DR_Sl', 
           'ACF_Plain_Footfall', 'ACF_Plain_Screened', 'ACF_Plain_Confirmed', 'ACF_Plain_Yield', 'ACF_Plain_TP', 
           'ACF_Plain_DS_Fl', 'ACF_Plain_DR_Fl', 'ACF_Plain_DR_Sl']
    
    
    m = ModelIntv()
    
    sel = rd.choice(list(range(len(pars_post))), n_collect)
    sel.sort()

    mss = list()
    mss_stats = list()


    for i in sel:
        pars = pars_post[i]
        y1, pars = m.find_baseline(pars, 2022)
    
        _, ms, _ = m.simulate_onward(y1, pars, intv={}, t_end=2031)

        ms = ms.reset_index().assign(Key=i, FollowUp=0, Duration=0)
        mss.append(ms)
        ms_stats = get_stats(ms, idx, Key=i, Scenario='Baseline', Coverage=coverage, FollowUp=0, Duration=0)
        mss_stats.append(ms_stats)
    
        for n_fu in np.linspace(0, 6, 5): # + [24, 30, 36]:
            if n_fu > 0:
                fu = 1 / n_fu
            else:
                fu = 0

            for c1 in np.linspace(0, 2, 20):
                intv = {'VulACF': {'Coverage': c1, 'FollowUp': fu, 'Duration': duration}}

                _, ms, _ = m.simulate_onward(y1, pars, intv=intv, t_end=2031)
                try:
                    ms = ms.reset_index().assign(Key = i, FollowUp=fu, Duration=duration)
                    mss.append(ms)
                    ms_stats = get_stats(ms, idx, Key=i, Scenario='Intervention', Coverage=c1, FollowUp=fu, Duration=duration)
                    mss_stats.append(ms_stats)
                except TypeError:
                    pass
            
    mss = pd.concat(mss)
    mss_stats = pd.DataFrame(mss_stats)
    return mss, mss_stats

In [7]:
for folder in ['dy_hi']:
    print(folder)
    out_path = f'out/{folder}'
    pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
    r_acf = json.load(open(f'{out_path}/R_ACF_Bg.json', 'r'))

    for p in pars_post:
        p.update(r_acf)
        
    mss, mss_stats = collect_fux(pars_post, n_collect=n_collect, coverage=0.03)
    mss.to_csv(f'{out_path}/Sim_VulACF_cex.csv')
    mss_stats.to_csv(f'{out_path}/Sim_VulACF_cex_stats.csv')

dy_hi
